# Корреляция Пирсона
Предположим, у нас есть выборка размера m и набор двух величин - $\{x\}_{i=1}^{m}$ и $\{y\}_{i=1}^{m}$. Задача - проверить, насколько коррелируют эти две измеряемые величины. В ноутбуке [проверка гипотез](minor-seminar-hypo.ipynb) мы уже научились проверять различные гипотезы, и такой подход часто можно использовать для проверки гипотез о принадлежности двух измеряемых величин (x и y) одному распределению. Однако, такой подход имеет следующий недостаток: x и y могут принадлежать разным распределениям, но сильно коррелировать между собой. 

Пример двух скоррелированных величин из двух разных распределений: $x\sim\mathcal{N(0,1)}$, $y=ax+b$

Оказывается, существует очень простой, наглядно интерпретируемый, эффективный и часто используемый подход для этого класса задача, называемый ***корреляцией Пирсона***. 

**Суть его очень проста:** 

Давайте посчитаем косинусную близость двух векторов: $X = \{x\}_{i=1}^{m}$ и $Y = \{y\}_{i=1}^{m}$ согласно указанным выше обозначениям. Подобное решение вполне мотивируемо - чем ближе x и y в каждом i'м измерении нашей выборки (i'я компонента нашего скалярного произведения m-мерных векторов), тем они скоррелированнее. Итоговая корреляция в этом простейшем случае получится $$\Sigma_{i=1}^mx_i*y_i = X\cdot Y$$

При таком подходе есть две проблемы. Первая заключается в том, что вектора x и y наших измерений могут иметь разное смещение (разное среднее). Это эквивалентно тому, что мы считаем косинусную близость между нашими векторами в m-мерном пространстве, исходящими из разных точек. Как-то неправильно. 

Пример такой ситуации: предположим, мы хотим посчитать корреляцию между оценками двух преподавателей, где каждое i-е измерение каждого ряда - это оценка соответствующего преподавателя i'му студенту. Мы заранее знаем, что оба препода адекватны, т.е. слабым студентам они ставят оценки ниже, а сильным - выше. Корреляция 100%. Однако один препод добрый и ставит оценки заведомо выше, чем второй. 

Исключить такого рода невязку очень просто - давайте **нормируем** $\{x\}_{i=1}^{m}$ и $\{y\}_{i=1}^{m}$ на их смещение (среднее). $$\Sigma_{i=1}^m(x_i-\bar{x})(y_i-\bar{y})=\bar{X}\cdot\bar{Y}$$

Огонь! Мы получили величину, называемую **ковариацией** $cov(X,Y)$

Однако, осталась ещё одна проблема - наши измерения могут иметь разной величины разброс, а значит, иметь разную длину. Пример: Один препод ставит "двоечникам" и "троечникам" 4, а всем остальным 5, в то время как второй ставит оценки от 0 до 10 в соответствии знаниям студента. В этом случае, даже если мы нормируем на среднее, мы будет считать косинусную близость между векторами разной длины в нашем m-мерном пространстве. Проблема опять же решается просто - давайте нормируем каждый наш вектор на длину:

$${{\bar{X}\cdot\bar{Y}}\over{||X||*||Y||}} = {{\Sigma_{i=1}^m(x_i-\bar{x})(y_i-\bar{y})}\over{\sqrt{\Sigma_{i=1}^m(x_i-\bar{x})^2}*\sqrt{\Sigma_{i=1}^m(y_i-\bar{y})^2}}}$$

Ништяк! Если теперь мы приглядимся к знаменателю, то это в точности выборочные дисперсии X и Y. Ну а в числителе всё та же ковариация. Вот эта величиная и называется ***Корреляция Пирсона***
$$cor(X,Y) = {{cov(X,Y)}\over{\sqrt{s_X^2 * s_Y^2}}}$$

In [1]:
import pandas as pd
import numpy as np

# Практическая задача

General Social Survey — ежегодный опрос населения США, записывается около 5000 факторов. Из опроса 2008 года выбраны данные о вере и образовании 2000 опрошенных. Есть ли связь между степенью веры и уровнем образования?

In [8]:
df = pd.DataFrame(data = [[9,  8,27, 8, 47,236],
                          [23,39,88,49,179,706],
                          [28,48,89,19,104,293]],
                 index=["Less than high school", "High school/junior college", "Bachelor/graduate"],
                 columns=["don't", "no way", "some power", "sometimes", "doubts", "know"])

In [12]:
from scipy import stats

correlations = {}

for education_1 in df.index:
    for education_2 in df.index:
        pearson_r, _ = stats.pearsonr(df.transpose()[education_1], df.transpose()[education_2])
        correlations[(education_1, education_2)] = pearson_r

correlations

{('Bachelor/graduate', 'Bachelor/graduate'): 1.0,
 ('Bachelor/graduate', 'High school/junior college'): 0.98344689290051468,
 ('Bachelor/graduate', 'Less than high school'): 0.98246780133204725,
 ('High school/junior college', 'Bachelor/graduate'): 0.98344689290051468,
 ('High school/junior college', 'High school/junior college'): 1.0,
 ('High school/junior college', 'Less than high school'): 0.99833884702465914,
 ('Less than high school', 'Bachelor/graduate'): 0.98246780133204725,
 ('Less than high school', 'High school/junior college'): 0.99833884702465914,
 ('Less than high school', 'Less than high school'): 1.0}

# Полезные ссылки
* Базовая [статья](http://www.machinelearning.ru/wiki/index.php?title=Коэффициент_корреляции_Пирсона) про корреляцию Пирсона
* Крутая [статья](https://habrahabr.ru/company/surfingbird/blog/139518/) про применение корреляции Пирсона в рекомендательных системах